In [64]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import time
from threading import Thread
from threading import Event
from SpArchClasses import MatrixAFetcher
from SpArchClasses import DistanceListBuilder
from SpArchClasses import MatrixBPrefetcher
from SpArchClasses import MultiplierArray
from SpArchClasses import MergeTree
from SpArchClasses import Scheduler
from SpArchClasses import Memory
from SpArchClasses import MatrixBCache

In [65]:
I = 10
K = 10
J = 10
NUM_INTS = 10

gen = np.random.default_rng()
data1 = gen.integers(1,10,NUM_INTS)
row1 = gen.integers(0,I,NUM_INTS)
col1 = gen.integers(0,K,NUM_INTS)

data2 = gen.integers(1,10,NUM_INTS)
row2 = gen.integers(0,K,NUM_INTS)
col2 = gen.integers(0,J,NUM_INTS)
i1 = csr_matrix(coo_matrix((data1, (row1, col1)), shape=(I, K)))
i2 = csr_matrix(coo_matrix((data2, (row2, col2)), shape=(K, J)))

In [66]:
print(i1.toarray())

[[0 0 3 7 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 2 0 0 0 0 0 0]
 [9 0 0 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 8 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 9 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [67]:
print(i2.toarray())

[[0 0 2 0 0 0 6 0 0 0]
 [0 0 8 0 0 0 0 0 0 0]
 [0 7 0 0 0 0 0 0 0 9]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 9 0 0 0 0 0 0 0 6]
 [0 0 6 0 0 0 0 0 0 2]
 [0 0 6 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [68]:
NUM_CHANNELS = 16
BANDWIDTH_PER_CHANNEL = 8
NUM_INPUT_FIFOS = 64
MERGE_SIZE = 16
MODE = "PrefetchCache"

AL = MatrixAFetcher(i1)
DL = DistanceListBuilder()
if MODE == "Cache":
    MBP = MatrixBCache(False,i2)
elif MODE == "PrefetchCache":
    MBP = MatrixBCache(True,i2)
else:
    MBP = MatrixBPrefetcher(i2)
MPA = MultiplierArray()
MT = MergeTree(MERGE_SIZE)
M = Memory(NUM_CHANNELS, BANDWIDTH_PER_CHANNEL)
S = Scheduler(AL,MT,DL,MBP,MPA,NUM_INPUT_FIFOS)

AL.setDLB(DL)
AL.setMemory(M)

DL.setMBP(MBP)
DL.setMultiplier(MPA)

MBP.setMultiplier(MPA)
MBP.setDistanceListBuilder(DL)
MBP.setMemory(M)

MPA.setMergeTree(MT)
MPA.setScheduler(S)

MT.setScheduler(S)

endFlag = True



In [69]:

ALEvent = Event()
ALEvent.set()
Thread(target=AL.running,args=[ALEvent]).start()

MBPEvent = Event()
MBPEvent.set()
Thread(target=MBP.running,args=[MBPEvent]).start()

MPAEvent = Event()
MPAEvent.set()
Thread(target=MPA.running,args=[MPAEvent]).start()

MTEvent = Event()
MTEvent.set()
Thread(target=MT.running,args=[MTEvent]).start()

count = 0
while endFlag:
    count += 1
    ALEvent.clear()
    MBPEvent.clear()
    MPAEvent.clear()
    MTEvent.clear()

    if not AL.endFlag:
        ALEvent.wait()
    if not MBP.endFlag:
        MBPEvent.wait()
    if not MPA.endFlag:
        MPAEvent.wait()
    if not MT.endFlag:
        MTEvent.wait()
    
    endFlag = not (AL.endFlag and MBP.endFlag and MPA.endFlag and MT.endFlag)
    
    if not MPA.endFlag:
        MPA.loadInputs()
    M.cycle()

# Cycle output
print("Cycle Count: ", count)

# Memory Bandwidth Utilization:
print("Average Bandwidth Utilization (%): " + str(M.TotalMemoryPulled/count) + ", Bandwidth Utilization When Memory Is In Use (%): " + str(M.TotalMemoryPulled/M.NumCyclesInUse))

# Memory Use
print("Memory Use ")
print("Matrix A Fetcher Memory Use (bytes): ", AL.memoryUse)
print("Matrix B Prefetcher Memory Use: (bytes)", MBP.memoryAccessBytes)
print("Total Memory Use: (bytes)", M.TotalMemoryPulled)

# Hardware Utilization
print("Hardware Utilization ")
print("Number of Rounds of Merging: ", S.rounds-1)
# We don't need any numbers for A matrix Fetcher. 
print("Matrix B Prefetcher Memory Wasted Cycles %: ", round(MBP.memoryWastedCycles/count,4), ", MBP Wasted Cycles: ", MBP.memoryWastedCycles)
print("Matrix B Prefetcher Hardware Utilization %: ", (1-round(MBP.wastedCycles/count,4)), ", MBP Wasted Cycles: ", MBP.wastedCycles)
print("Multiplier Array Hardware Utilization %", (1-round(MPA.wastedCycles/count,4)), ", MPA Wasted Cycles: ", MPA.wastedCycles)
print("MergeTree Cycles Idle (No merging) %", (round(MT.idleCycles/count,4)) ,", MergeTree Cycles Idle: ", MT.idleCycles)
print("MergeTree Wasted Cycles (No Partial Loading or Merging) %", (round(MT.wastedCycles/count,4)) ,", MergeTree Cycles Wasted: ", MT.wastedCycles)
print("MergeTree Average FIFO Uptime (%)", (1-round(MT.totalEmptyFifos/MT.totalMergeCycles,4)))

1
1
Cycle Count:  283
Average Bandwidth Utilization (%): 8.424028268551236, Bandwidth Utilization When Memory Is In Use (%): 29.8
Memory Use 
Matrix A Fetcher Memory Use (bytes):  80
Matrix B Prefetcher Memory Use: (bytes) 2304
Total Memory Use: (bytes) 2384
Hardware Utilization 
Number of Rounds of Merging:  1
Matrix B Prefetcher Memory Wasted Cycles %:  0.4982 , MBP Wasted Cycles:  141
Matrix B Prefetcher Hardware Utilization %:  0.5406 , MBP Wasted Cycles:  130
Multiplier Array Hardware Utilization % 0.5583 , MPA Wasted Cycles:  125
MergeTree Cycles Idle (No merging) % 0.0035 , MergeTree Cycles Idle:  1
MergeTree Wasted Cycles (No Partial Loading or Merging) % 0.5512 , MergeTree Cycles Wasted:  156
MergeTree Average FIFO Uptime (%) 0.34330000000000005


In [70]:
data = []
i = []
j = []
for x in MT.partialMatrices[-1]:
    data.append(x[0])
    i.append(x[1])
    j.append(x[2])

total = coo_matrix((data,(i,j)),shape=(I,K))
print(total.toarray())
print(np.matmul(i1.toarray(),i2.toarray()))
print(np.equal(total.toarray(), np.matmul(i1.toarray(),i2.toarray())))
print(np.allclose(total.toarray(),np.matmul(i1.toarray(),i2.toarray()),rtol=0.01))


[[ 0 21  0  0  0  0  0  0  0 27]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0 54  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 48  0  0  0  0  0  0 16]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 36  0  0  0  0  0  0 12]
 [ 0  0  0  0  0  0  0  0  0  0]]
[[ 0 21  0  0  0  0  0  0  0 27]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0 54  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 48  0  0  0  0  0  0 16]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0 36  0  0  0  0  0  0 12]
 [ 0  0  0  0  0  0  0  0  0  0]]
[[ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  Tr